In [1]:
begin
	using Random
	using Distributions
	using Plots
	using LinearAlgebra
end

In [2]:
begin 
	seed = 1234
	Random.seed!(1234)
end

TaskLocalRNG()

In [3]:
begin
	
	#define aminoacids according to the standard one letter abbreviation and assign 
	#them to a number
	@enum Aminoacid begin
		A = 1
		R = 2
		N = 3
		D = 4
		C = 5
		E = 6
		Q = 7
		G = 8
		H = 9
		I = 10
		L = 11
		K = 12
		M = 13
		F = 14
		P = 15
		S = 16
		T = 17
		W = 18
		Y = 19
		V = 20
	end	
	
	interaction_mat = rand(Uniform(-4.00, -2.00), 20, 20);
	interaction_mat = tril(interaction_mat)
	interaction_mat += interaction_mat' - Diagonal(diag(interaction_mat))
	heatmap(interaction_mat, colormap=:jet)
	
end

# ╔═╡ 36716b00-e7ba-11ee-119e-0ffc0a40ee38
mutable struct Monomer
	id::Int64
	kind::Aminoacid #type of the monomer
	nearest_neighbors::Vector{Monomer}
	pos::Vector{Int64}
	available_moves::Vector{Vector{Int64}}
	function Monomer(id::Int64, kind::Aminoacid ,pos::Vector{Int64})
		new(id, kind, Vector{Monomer}(), pos)
	end
end

In [4]:
function isdistance1(pos1, pos2)
    return norm(pos1 - pos2) <= 1
end

isdistance1 (generic function with 1 method)

In [5]:
function randPolymer(init::Vector{Int64}, N::Int64)::Vector{Monomer}
	
	aminos = [Aminoacid(rand(1:20)) for _ ∈ 1:N]
	monomers = [Monomer(1, aminos[1], init)]
	
	for i in 2:N
		maxiter = 2
		new_pos = Int64[]
		isValid = false
		
		while !isValid && maxiter < 1000
			
			idx = rand([1,2])
			direction = rand([-1,1])
			move = zeros(2)
			move[idx] =  direction
			new_pos = monomers[i-1].pos + move	
			
			isValid = all(norm(new_pos - m.pos)!=0 for m ∈ monomers)
			isValid *= all(new_pos .> 0)
			maxiter+=1
			
		end
		
		if maxiter==1000
			error("Failed to build a polymer")
			break
		end
		
		push!(monomers, Monomer(i, aminos[i], Int64.(new_pos)))
	
	end
	return monomers
end

randPolymer (generic function with 1 method)

In [6]:
function straightPolymer(init::Vector{Int64}, N::Int64)::Vector{Monomer}
		
    aminos = [Aminoacid(rand(1:20)) for _ ∈ 1:N]
    monomers = [Monomer(1, aminos[1], init)]

    for i in 2:N
        new_pos = monomers[i-1].pos + [1,0]
        push!(monomers,Monomer(i, aminos[i],new_pos))
    end
    return monomers
end

straightPolymer (generic function with 1 method)

In [7]:
mutable struct Polymer
	N::Int64 #number of monomers in the structure
	monomers::Vector{Monomer} #constituents of the polymer
	energy::Float64
	grid::Matrix{Int64}
	is_movable::Vector{Int64}
	function Polymer(monomers::Vector{Monomer})
		
		N = length(monomers)
		
		grid = zeros(Int64, 4*N,4*N) 
		energy = 0.00
		for m ∈ monomers
			grid[m.pos[1], m.pos[2]] = m.id
		end
		for m ∈ monomers
			x,y = m.pos
			
			indices = [(x+1, y),(x-1, y),(x, y+1),(x, y-1)]
			
			pot_neighbors = filter(n-> n != 0, [grid[i,j] for (i,j) ∈ indices])
			m.nearest_neighbors = monomers[pot_neighbors]
			if m.id > 1
				# Remove the previous monomer (before) from the nearest_neighbors
				m.nearest_neighbors = filter(n -> n != monomers[m.id - 1], 											 m.nearest_neighbors)
			end
			if m.id < length(monomers)
				# Remove the next monomer (after) from the nearest_neighbors
				m.nearest_neighbors = filter(n -> n != monomers[m.id + 1], 											 m.nearest_neighbors)
			end
			for n ∈ m.nearest_neighbors
				energy += interaction_mat[Int(m.kind),Int(n.kind)]
			end
		end
		energy = energy/2        #/2 to account for double counting of the energy
		new(N,monomers,energy, grid, Vector{Int64}())
	end
end

In [8]:
function availableMoves!(mm::Monomer, pm::Polymer)
    nrows, ncols = size(pm.grid)
	row,col = mm.pos

	# Offsets for Up, Down, Left, Right, Diagonals
	offsets = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1),
				(-2, 0), (2, 0), (0, -2), (0, 2)]

    neighbors = [
        [row + dr, col + dc] for (dr, dc) in offsets
        if 1+1 <= row + dr <= nrows-1 && 1+1 <= col + dc <= ncols-1 #Remember to fix
				&& pm.grid[row + dr, col + dc] == 0 ]
	if mm.id > 1
		neighbors = filter(n->isdistance1(n, pm.monomers[mm.id-1].pos), neighbors)
	end
	if mm.id < length(pm.monomers)
		neighbors = filter(n->isdistance1(n, pm.monomers[mm.id+1].pos), neighbors)
	end
	mm.available_moves = neighbors
	if !isempty(neighbors)
		push!(pm.is_movable, mm.id)
	end 
end

availableMoves! (generic function with 1 method)

In [9]:
function updateMoves!(pm::Polymer)
	resize!(pm.is_movable, 0)
	for m in pm.monomers
		availableMoves!(m,pm)
	end
end


updateMoves! (generic function with 1 method)

In [10]:
function moveMonomer!(pm::Polymer, m::Monomer, pos_new::Vector{Int64})
	
	pm.monomers[m.id].pos = pos_new
	pm.grid[m.pos[1], m.pos[2]] = 0
	pm.grid[pos_new[1], pos_new[2]] = m.id

	old_neighbors = copy(m.nearest_neighbors)
	resize!(m.nearest_neighbors, 0)

	x,y = m.pos
	
	indices = [(x+1, y),(x-1, y),(x, y+1),(x, y-1)]
	pot_neighbors = filter(n-> n != 0, [pm.grid[i,j] for (i,j) ∈ indices])
	m.nearest_neighbors = pm.monomers[pot_neighbors]
	
	push!([nm for nm ∈ m.nearest_neighbors], m)
	if m.id > 1
		# Remove the previous monomer (before) from the nearest_neighbors
		m.nearest_neighbors = filter(n -> n != pm.monomers[m.id - 1], 											 m.nearest_neighbors)
	end
	if m.id < length(pm.monomers)
		# Remove the next monomer (after) from the nearest_neighbors
		m.nearest_neighbors = filter(n -> n != pm.monomers[m.id + 1], 											 m.nearest_neighbors)
	end
	setdiff!(old_neighbors, m.nearest_neighbors)
	for nm ∈ old_neighbors
		idx = findfirst(n -> n === m, pm.monomers[nm.id].nearest_neighbors)
	    if idx !== nothing
	        deleteat!(pm.monomers[nm.id].nearest_neighbors, idx)
	    end
	end
	pm.monomers[m.id] = m
end

moveMonomer! (generic function with 1 method)

In [11]:
function calculateEnergy!(pm::Polymer)

	monomers = pm.monomers
	energy = 0.0

	for m ∈ monomers	
		for n ∈ m.nearest_neighbors
			energy += interaction_mat[Int(m.kind),Int(n.kind)]
		end
	end 
	pm.energy = energy/2
end 

calculateEnergy! (generic function with 1 method)

In [12]:
function calculateEndToEnd(pm)
	return norm(pm.monomers[end].pos-pm.monomers[1].pos)
end

calculateEndToEnd (generic function with 1 method)

In [13]:
function calculateRoG(pm)
	coordinates = hcat([monomer.pos for monomer ∈ pm.monomers]...)
    center_of_mass = sum(coordinates, dims=1) ./ size(coordinates, 1)
    squared_distances = sum((coordinates .- center_of_mass).^2, dims=2)
    return sqrt(sum(squared_distances) / size(coordinates, 1))
end

calculateRoG (generic function with 1 method)

In [14]:
function plotPolymer!(plt, pm, i)
	mm = deepcopy(pm.monomers) # we dont change the original system, just the plot 
	for monomer ∈ mm
		
	    # Plot lines to the nearest neighbors for each monomer
	    for neighbor ∈ monomer.nearest_neighbors
	        plot!(plt, [monomer.pos[1], neighbor.pos[1]],
	              [monomer.pos[2], neighbor.pos[2]],
	              color = :green, line=(:dash, [2,2]) )
 
			#removes the current monomer from the list with nearest neighbors of the its own nearest neighbors to avoid double plotting the dashed line
			filter!(n -> !(n==monomer), neighbor.nearest_neighbors)
				 
	    end 
	end
	x_positions = [monomer.pos[1] for monomer ∈ pm.monomers] 
	y_positions = [monomer.pos[2] for monomer ∈ pm.monomers]

	
	# Create a scatter plot of the positions
	plot!(plt, x_positions, y_positions, linecolor="black", linewidth=4, alpha = 1)
	scatter!(plt, x_positions[1:end], y_positions[1:end], 
			color=:red, markersize=5, legend=false, title = "After $i sweeps") 
	scatter!(plt, [x_positions[1]], [y_positions[1]], 
			color=:orange, markersize=5, legend=false)
end


plotPolymer! (generic function with 1 method)

In [15]:
function Boltzmann(E_new::Float64, E_old::Float64, T::Float64)::Float64
	ΔE = E_new - E_old
	return exp(-ΔE/T)
end 


Boltzmann (generic function with 1 method)

In [16]:
struct Log 
	steps::Int64
	energy::Float64
	end_to_end::Float64
	RoG::Float64
	accepted::Int64
	rejected::Int64
	function Log(steps::Int64, energy::Float64,end_to_end::Float64,
				RoG::Float64, accepted::Int64, rejected::Int64)
		new(steps, energy, end_to_end, RoG, accepted, rejected)
	end
end

In [17]:
struct Logger
	header::String
	logs::Vector{Log}
	footer::String
	file::String
	function Logger(seed::Int64,temperature::Float64,N::Int64, file)
		head = "Starting MC simulation \nSeed: $(seed) \nTemperature: $(temperature)\nNumber of monomers: $(N)\nSteps Energy End-to-end RoG Accepted Rejected\n\n"
		foot = "\n Simulation completed\n\n"
		new(head, Vector{Log}(), foot, file)
	end
end

In [18]:
function addLog!(logger::Logger, steps::Int64, energy::Float64, 
    end_to_end::Float64, RoG::Float64, accepted::Int64, rejected::Int64)
push!(logger.logs, Log(steps, energy, end_to_end, RoG, accepted, rejected))
end


addLog! (generic function with 1 method)

In [19]:
function writeLog(logger::Logger)
	filename = "raw/" * logger.file
	io = open(filename, "w+")
	write(io, logger.header)
	for log ∈ logger.logs
    	write(io, "$(log.steps),$(log.energy),$(log.end_to_end),$(log.RoG),$(log.accepted),$(log.rejected)\n")
	end
    write(io, logger.footer)
	flush(io)
	close(io)
end

writeLog (generic function with 1 method)

In [20]:
const temperature10 = 10.0

10.0

In [21]:
function runAndPlotMC!(pm::Polymer, steps::Int64, distribution::Function, 
    temperature::Float64; seed=123, filelog="MC_log.txt")

logger = Logger(seed, temperature, length(pm.monomers), filelog)

updateMoves!(pm) 	

accepted = 0
rejected = 0

energies = Float64[]


push!(energies, pm.energy)

end_to_end = calculateEndToEnd(pm)
RoG = calculateRoG(pm)
addLog!(logger, 0, pm.energy, end_to_end, RoG, 0, 0)

plt = plot(layout = 4, grid = false)

plotPolymer!(plt[1], pm, 0)

for i ∈ 1:steps
for j ∈ 1:length(pm.monomers)
pm_old = deepcopy(pm)

monomer = pm.monomers[rand(pm.is_movable)]
new_pos = rand(monomer.available_moves)
moveMonomer!(pm, monomer, new_pos)

pm_new= Polymer(pm.monomers)

calculateEnergy!(pm_new)

accept = false 

if pm_old.energy >= pm_new.energy
accept = true
else
b = Boltzmann(pm_new.energy, pm_old.energy, temperature)
if b >= rand()
accept = true
else
accept = false
end
end
if accept
accepted+=1
updateMoves!(pm_new)
calculateEnergy!(pm_new) 
pm = deepcopy(pm_new) 
else
rejected+=1
updateMoves!(pm_old)
calculateEnergy!(pm_old) 
pm = deepcopy(pm_old)
end
end
if i % 10 == 0
push!(energies, pm.energy)
end_to_end = calculateEndToEnd(pm)
RoG = calculateRoG(pm)
addLog!(logger, i, pm.energy, end_to_end, RoG, accepted, rejected)
end
if i == 10
plotPolymer!(plt[2], pm, i)
end
if i == 100
plotPolymer!(plt[3], pm, i)
end
if i == steps
plotPolymer!(plt[4], pm, i)
end
end 
writeLog(logger)
return energies, plt
end

runAndPlotMC! (generic function with 1 method)

In [22]:
function MCstep!(pm::Polymer, accepted::Int, rejected::Int, distribution::Function, T::Float64)
	
	for j ∈ 1:length(pm.monomers)
		pm_old = deepcopy(pm)

		monomer = pm.monomers[rand(pm.is_movable)]
		new_pos = rand(monomer.available_moves)
		moveMonomer!(pm, monomer, new_pos)

		pm_new= Polymer(pm.monomers)
		
		calculateEnergy!(pm_new)
		
		accept = false 

		if pm_old.energy >= pm_new.energy
			accept = true
		else
			b = distribution(pm_new.energy, pm_old.energy, T)
			if b >= rand()
				accept = true
			else
				accept = false
			end
		end
		if accept
			accepted+=1
			updateMoves!(pm_new)
			calculateEnergy!(pm_new) 
			pm = deepcopy(pm_new) 
		else
			rejected+=1
			updateMoves!(pm_old)
			calculateEnergy!(pm_old) 
			pm = deepcopy(pm_old)
		end
	end
	return pm
end

# ╔═╡ c9ba9a81-7491-48ff-86cd-644e18fcf2a0
function MonteCarlo!(pm::Polymer, steps::Int64, distribution::Function, T::Float64; 						seed=1234, filelog="MC_log.txt")
	
	logger = Logger(seed, T, length(pm.monomers), filelog)
	
	updateMoves!(pm) 	
	
	accepted = 0
	rejected = 0
	
	energies = Float64[]
	

	push!(energies, pm.energy)

	end_to_end = calculateEndToEnd(pm)
	RoG = calculateRoG(pm)
	addLog!(logger, 0, pm.energy, end_to_end, RoG, 0, 0)
	

	for i ∈ 1:steps
		pm = MCstep!(pm, accepted, rejected, distribution, T)
		if i % 10 == 0
			push!(energies, pm.energy)
			end_to_end = calculateEndToEnd(pm)
			RoG = calculateRoG(pm)
			addLog!(logger, i, pm.energy, end_to_end, RoG, accepted, rejected)
		end

	end 
	
	return pm, logger
end

MonteCarlo! (generic function with 1 method)

In [23]:
function initiate(N::Int64, T::Float64; seed=1234)
	iter = 0
	pm = Polymer(straightPolymer([20,20], N))
	updateMoves!(pm) 	

	while iter<50 || abs(pm.energy) > 1e-9
		iter+=1 
		pm = MCstep!(pm, 0, 0, Boltzmann, T)
	end
	return pm
end


initiate (generic function with 1 method)

In [24]:
begin 
	polymer = initiate(15, 100.0)#Polymer(straightPolymer([20,20], 15))
	energies, plt = runAndPlotMC!(polymer, 10000, Boltzmann, 10.0; seed=seed) 
end

([0.0, -2.603044561910255, -15.378285496046981, -8.105840949512483, -9.664915679393019, 0.0, -2.3800110528287775, -9.641942985211964, -7.839434498979967, -2.208289969175694  …  -11.406820577088252, -5.081725503174836, -14.517702983110016, -18.071384359856534, -8.323475800067982, -12.619740186142453, -14.873388061965265, -6.156725938656772, -20.24619254004005, -2.1748081801835135], Plot{Plots.GRBackend() n=14})

In [25]:
function running_avg(x)
	x_avg = cumsum(x)
	for i ∈ 1:length(x)
		x_avg[i] = x_avg[i]/(i)
	end
	return x_avg, 0:10:10*length(x)-1
end

running_avg (generic function with 1 method)

In [ ]:
begin
	energies_avg, steps =  running_avg(energies)
	plt2 = plot()
	plot!(steps, energies_avg, 
		xlabel = "Number of sweeps", ylabel = "Average energy [\$k_B\$K]", 
		label = "\$T = $(temperature10)\$", linewidth = 2)
end

In [29]:
begin
	pm = initiate(20, 100.0)	
	pm = MonteCarlo!(pm, 500, Boltzmann, 10.0;
							seed=seed, filelog = "annealing/MC.txt")[1]
	energies_avg3, steps3 =  running_avg([l.energy for l ∈ logger.logs])
	plt3 = plot()
	plot!(steps3, energies_avg3)
end

UndefVarError: UndefVarError: `logger` not defined

In [27]:
function annealing(N::Int64)
	T_arr = 40:-1:1
	steps = 2000
	pm = initiate(N, 100.0)	
    Threads.@threads for t in T_arr
		pm, logger = MonteCarlo!(pm, steps, Boltzmann, Float64(t);
						seed=seed, filelog = "annealing/MC_$(N)x$(steps)_$(t)a.txt")
		writeLog(logger)
	end
end

annealing (generic function with 1 method)

In [28]:
annealing(10) 